# Dask Experiments

This notebook showcases the Dask implementation and experiments.


In [1]:

from erpub.pipeline.preprocessing_dask import all_lowercase_and_stripped_dask
from experiment_utils import (
    plot_blocking_comparison,
    evaluate_blocking_method,
    plot_matching_accs,
    get_accuracy_of_matches,
)
import numpy as np
from erpub.pipeline.pipeline_dask import DaskPipeline
from erpub.pipeline.matching_dask import jaccard_similarity, specific_name_matcher_dask
from dask.distributed import Client


In [3]:
def run_matching_experiment(pipeline, name, thresholds, plot=True):
    accs = []
    for threshold in thresholds:
        pipeline.run(f"experiments/{name}_threshold_{threshold}", threshold)
        accs.append(get_accuracy_of_matches(f"experiments/{name}_threshold_{threshold}/matched_entities.csv"))
    if plot:
        plot_matching_accs(f"Matching accuracy of experiment: {name}", thresholds, accs)
    print(f"Matching accuracys: {accs}")

We're starting with a simple setup with jaccard_similarity and data preprocessing.

Disclaimer: Unfortunately there are still pickle serialization issues, impacting part of the functionality

In [4]:
global client, pipe, ddf
client = Client()
pipe = DaskPipeline("data/prepared/", client=client, matching_fns=jaccard_similarity,
                    similarity_threshold=0.8, preprocess_data_fn=all_lowercase_and_stripped_dask)
ddf = pipe.run("experiments", np.linspace(0.3, 0.9, num=5))

2024-02-02 21:58:26,175 - The pipeline will be built with these files: ['data/prepared/ACM_1995_2004_rep_9x.csv', 'data/prepared/DBLP_1995_2004_rep_4x.csv', 'data/prepared/ACM_1995_2004_rep_7x.csv', 'data/prepared/DBLP_1995_2004_rep_6x.csv', 'data/prepared/DBLP_1995_2004_rep_8x.csv', 'data/prepared/DBLP_1995_2004_rep_3x.csv', 'data/prepared/ACM_1995_2004_rep_4x.csv', 'data/prepared/DBLP_1995_2004_rep_9x.csv', 'data/prepared/DBLP_1995_2004_rep_10x.csv', 'data/prepared/DBLP_1995_2004_rep_7x.csv', 'data/prepared/ACM_1995_2004_rep_8x.csv', 'data/prepared/DBLP_1995_2004.csv', 'data/prepared/ACM_1995_2004.csv', 'data/prepared/ACM_1995_2004_rep_6x.csv', 'data/prepared/DBLP_1995_2004_rep_5x.csv', 'data/prepared/DBLP_1995_2004_rep_2x.csv', 'data/prepared/ACM_1995_2004_rep_3x.csv', 'data/prepared/ACM_1995_2004_rep_5x.csv', 'data/prepared/ACM_1995_2004_rep_10x.csv', 'data/prepared/ACM_1995_2004_rep_2x.csv']
2024-02-02 21:58:27,569 - Loaded csv successfully into Dask dataframe
2024-02-02 21:58:27,

TypeError: ('Could not serialize object of type HighLevelGraph', '<ToPickle: HighLevelGraph with 28 layers.\n<dask.highlevelgraph.HighLevelGraph object at 0x7ff6ac286910>\n 0. to_pyarrow_string-ca5ad36d3ac56196ffac587b4dbb6551\n 1. to_pyarrow_string-9a92e43894348896cef5608546fd2a72\n 2. to_pyarrow_string-a7472f2d73f9551a46e10a70c3bfe777\n 3. to_pyarrow_string-53f887dd3bafa991bd923bd1156f8e7b\n 4. to_pyarrow_string-f5bcdd2af0c4269025256ac2efa242d3\n 5. to_pyarrow_string-2f205110f49eb1bb31d93dc4556c034b\n 6. to_pyarrow_string-435a3d7101b7a019301a1eb579413a25\n 7. to_pyarrow_string-c417ee8820c70dbdfb729b68c0af8274\n 8. to_pyarrow_string-0277849ef86ac7128c2de66b781c5786\n 9. to_pyarrow_string-6dd202d086595101e1e3ecd982473e1c\n 10. to_pyarrow_string-0f3e36ec8b0bc7101679c0b1e4045403\n 11. to_pyarrow_string-6f60988dd003135f67bd26c2af27dead\n 12. to_pyarrow_string-b805aa60394bc3ed7eec68dec1146232\n 13. to_pyarrow_string-86a2e9055c0155746ffa580783f7348a\n 14. to_pyarrow_string-452ab10006bc6f68c

To view the Dask dashboard and get an overview over running workers and jobs, click the link below:

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32831,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.67 GiB
Comm: tcp://127.0.0.1:35301,Total threads: 2
Dashboard: http://127.0.0.1:36395/status,Memory: 1.92 GiB
Nanny: tcp://127.0.0.1:37631,


## Advanced pipeline
Our best performing configuration, now with Dask

In [6]:
from erpub.pipeline.matching_dask import jaccard_similarity
from erpub.pipeline.preprocessing_dask import all_lowercase_and_stripped_dask

In [ ]:
compare_to_baseline(
    blocking_fn=author_names_initials,
    matching_fns={"paper_title": jaccard_similarity,
                  "year_of_publication": equality_matcher},
    thresholds=thresholds,
)

In [ ]:
pipeline = DaskPipeline(
    file_dir="data/prepared/",
    preprocess_data_fn=all_lowercase_and_stripped_dask,
    blocking_fn=author_names_initials,
    matching_fns={
        "paper_title": jaccard_similarity,
        "year_of_publication": equality_matcher,
    },
    verbose=False,
)
run_matching_experiment(pipeline, "author_names_initials_blocking", thresholds)